**Домашнее задание 3**

В первой части третьего дз предлагается поработать с несколькими датасетами из соревнования [Boston Airbnb Open Data на Kaggle](https://www.kaggle.com/airbnb/boston). Данные можно найти на Kaggle, или по  [ссылке](https://drive.google.com/drive/folders/1lTAxxEwrCkd5WEQMJCJvBsFZc3tXgiyO?usp=sharing). Решите предложенные задания с использованием модуля Pandas (NumPy, разумеется, тоже разрешен).

Использование условий и циклов запрещено (списковые включения это тоже цикл!), за это будут обнуляться баллы.
Все задания равноценны (кроме нулевого, его выполнение необходимо для получения положительной оценки).
В каждой задаче постарайтесь придумать несколько решений. Если вы напишете несколько решений (или несколько способов сделать какую-то часть решения), сравните их производительность с помощью jupyter magics. Решением должен быть Jupyter notebook / ссылка на Colab

In [1]:
import pandas as pd
import numpy as np

listings = pd.read_csv('listings.csv')
calendar = pd.read_csv('calendar.csv')
reviews = pd.read_csv('reviews.csv')

pd.set_option('display.max_columns', None)

0. Внимательно посмотрите на данные в таблицах. При попытке посмотреть первые несколько строк в таблице listings с помощью pandas отображаются не все колонки. Сделайте так, чтобы отображались все столбцы

    Для всех ли объявлений есть календарь? Для какой доли объявлений нет ни одного отзыва? Сколько объявлений не были доступны для сдачи ни разу за представленный период?

In [3]:
len_listings, len_calendar = listings['id'].nunique(), calendar['listing_id'].nunique()
print(f"Все ли объявления имеют календарь? {(len_listings == len_calendar)}")

Все ли объявления имеют календарь? True


Да

In [7]:
print('Доля объявлений  отзывов: ', len(listings[~listings['id'].isin(reviews['listing_id'])]) / len_listings * 100, '%', sep = "")

Доля объявлений  отзывов: 21.08786610878661%


In [9]:
print('Количество объявлений, которые никогда не были доступны оступны для сдачи ни разу за представленный период:', calendar.groupby('listing_id')['available'].apply(lambda x: (x == 'f').all()).sum())

Количество объявлений, которые никогда не были доступны оступны для сдачи ни разу за представленный период: 679


1. Как зовут владельца, у недвижимости которого за отчетный период была наибольшая разница между максимальной и минимальной ценой?

In [12]:
listings[:5]

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",NaN,"The bus stop is 2 blocks away, and frequent. B...","You will have access to 2 bedrooms, a living r...",NaN,Clean up and treat the home the way you'd like...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,31303940,https://www.airbnb.com/users/show/31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",We are country and city connecting in our deck...,NaN,NaN,NaN,f,https://a2.muscache.com/im/pictures/5936fef0-b...,https://a2.muscache.com/im/pictures/5936fef0-b...,Roslindale,1,1,"['email', 'phone', 'facebook', 'reviews']",t,f,"Birch Street, Boston, MA 02131, United States",Roslindale,Roslindale,NaN,Boston,MA,02131,Boston,"Boston, MA",US,United States,42.282619,-71.133068,t,House,Entire home/apt,4,1.5,2.0,3.0,Real Bed,"{TV,""Wireless Internet"",Kitchen,""Free Parking ...",NaN,$250.00,NaN,NaN,NaN,$35.00,1,$0.00,2,1125,2 weeks ago,NaN,0,0,0,0,2016-09-06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...","If you don't have a US cell phone, you can tex...",Plenty of safe street parking. Bus stops a few...,Apt has one more bedroom (which I use) and lar...,"If I am at home, I am likely working in my hom...",Pet friendly but please confirm with me if the...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,2572247,https://www.airbnb.com/users/show/2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",I live in Boston and I like to travel and have...,within an hour,100%,100%,f,https://a2.muscache.com/im/users/2572247/profi...,https://a2.muscache.com/im/users/2572247/profi...,Roslindale,1,1,"['email', 'phone', 'facebook', 'linkedin', 'am...",t,t,"Pinehurst Street, Boston, MA 02131, United Sta

In [14]:
listings['price'] = listings['price'].replace('[\$,]', '', regex=True).astype(float)
df = listings.groupby(['name']).agg({
    'price': ['max', 'min']
})
df['price_diff'] = df['price']['max'] - df['price']['min']
df[df['price_diff'] != 0]

price        price_diff
                                           max    min           
name                                                            
1 br Walking Distance to Fenway          239.0  209.0       30.0
14 Gloucester Street by Lyon Apartments  296.0  256.0       40.0
1BR West End Apt. near Restaurants       450.0  189.0      261.0
Beautiful Beacon Hill 1 Bedroom          275.0  150.0      125.0
Charming South End Apartment             200.0  195.0        5.0
Chinatown Single Bedroom                  95.0   93.0        2.0
Clean Spacious 1 BR Near T Station        60.0   55.0        5.0
Comfortable Brownstone Home               79.0   73.0        6.0
Cozy guest room. Awesome location!        69.0   65.0        4.0
Cozy modern room by the T                 80.0   75.0        5.0
Downtown Crossing, Studio Apt, Boston    239.0  229.0       10.0
Dowtown Boston 2BR Apt.-Opera House      299.0  199.0      100.0
Furnished Luxury 2BR at West End         249.0  199.0       50.0
Lux 3BR in Seaport Square w/wifi         506.0  389.0      117.0
Luxury 1BR in Boston Fenway              229.0  169.0       60.0
Marriott Custom House                    400.0  250.0      150.0
North End Gem                            350.0  145.0      205.0
Private Room in Beacon Hill              140.0  120.0       20.0
Private Room in the Heart of Boston      115.0   90.0       25.0
Private cozy room                         69.0   45.0       24.0
Private room in Boston                    80.0   52.0       28.0
Quarters on Dot - hostel style unit      130.0  105.0       25.0
South End Condo                          249.0  190.0       59.0
Spacious modern 2BR apt by the T         200.0  175.0       25.0
Spectacular Beacon Hill 1 bedroom        229.0  210.0       19.0
Stay with Fun Family                      48.0   45.0        3.0
The Treehouse                            145.0  140.0        5.0
The apartment you were looking for!      225.0  125.0      100.0
West Broadway Quarters - hostel unit     220.0  144.0       76.0

In [16]:
df = df.reset_index()
print('Имя владельца:', df.loc[df['price_diff'].idxmax()]['name'].iloc[0])

Имя владельца: 1BR West End Apt. near Restaurants


2. В таблице listings достаточно много самых разных колонок. В частности, есть host_listings_count, host_total_listings_count и calculated_host_listings_count. По названию понятно, что это "какое-то" число объявлений хозяина недвижимости, но какое?

    Внимание вопрос. Является ли какое-то из этих чисел количеством объявлений из listings для каждого хозяина недвижимости? Сколько в среднем объектов недвижимости у хозяина из числа указанных в listings? А сколько максимально?

    Приветствуются промежуточные вычисления и рассуждения по этому вопросу. Например, было бы круто проверить, что для всех записей одного и того же хозяина одинаковые значения в указанных выше трех столбцах

In [19]:
df_merged = listings.merge(listings.groupby('host_id').size().reset_index(name='listings_count'), on='host_id', how='left')
df_merged[['host_id', 'host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count', 'listings_count']].drop_duplicates().head(10)

,host_id,host_listings_count,host_total_listings_count,calculated_host_listings_count,listings_count
0,31303940,1,1,1,1
1,2572247,1,1,1,1
2,16701,1,1,1,1
3,6031442,1,1,1,1
4,15396970,1,1,1,1
5,64200298,2,2,1,1
6,6570877,5,5,3,3
7,3164508,2,2,2,2
8,3962517,1,1,1,1
9,4434391,2,2,2,2


In [21]:
avr_listings = df_merged['listings_count'].mean()
max_listings = listings['host_listings_count'].max()
print('Среднее количество объектов недвижимости у хозяина:', avr_listings)
print('Максимальное количество объектов недвижимости у хозяина:', max_listings)

Среднее количество объектов недвижимости у хозяина: 12.733891213389121
Максимальное количество объектов недвижимости у хозяина: 749


In [23]:
result = df_merged.groupby('host_id').agg({'host_listings_count': 'nunique', 'host_total_listings_count': 'nunique', 'calculated_host_listings_count': 'nunique'})
print((result == 1).all())

host_listings_count               True
host_total_listings_count         True
calculated_host_listings_count    True
dtype: bool


для всех записей одного и того же хозяина одинаковые значения в указанных выше трех столбцах

In [26]:
df_merged['host_listings_count_match'] = (df_merged['host_listings_count'] == df_merged['listings_count'])
df_merged['host_total_listings_count_match'] = (df_merged['host_total_listings_count'] == df_merged['listings_count'])
df_merged['calculated_host_listings_count_match'] = (df_merged['calculated_host_listings_count'] == df_merged['listings_count'])
df_merged[['host_id', 'host_listings_count_match', 'host_total_listings_count_match', 'calculated_host_listings_count_match']].drop_duplicates().head(10)

,host_id,host_listings_count_match,host_total_listings_count_match,calculated_host_listings_count_match
0,31303940,True,True,True
1,2572247,True,True,True
2,16701,True,True,True
3,6031442,True,True,True
4,15396970,True,True,True
5,64200298,False,False,True
6,6570877,False,False,True
7,3164508,True,True,True
8,3962517,True,True,True
9,4434391,True,True,True


In [28]:
print(df_merged['calculated_host_listings_count_match'].all())

True


Является ли какое-то из этих чисел количеством объявлений из listings для каждого хозяина недвижимости? Да, calculated_host_listings_count_match. Сколько в среднем объектов недвижимости у хозяина из числа указанных в listings? 12.733891213389121 А сколько максимально? 749

3. Коррелируют ли средняя цена на недвижимость за всё время наблюдений и количество отзывов на неё? Иными словами, если средняя цена квартиры больше, то на неё оставляют больше или меньше отзывов?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

listings['price'] = listings['price'].replace('[\$,]', '', regex=True).astype(float)
calendar['price'] = calendar['price'].replace('[\$,]', '', regex=True).astype(float)
cnt = reviews.groupby('listing_id').size().reset_index(name = 'cnt')
df_merged = calendar.groupby('listing_id')['price'].mean().reset_index().merge(cnt, on='listing_id', how='left')
df_merged['cnt'] = df_merged['cnt'].fillna(0).astype(int)
print('Корреляция между средней ценой и количеством отзывов:',  df_merged['price'].corr(df_merged['cnt']))

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_merged, x='price', y='cnt', alpha=0.5)
plt.title('Корреляция между средней ценой и количеством отзывов')
plt.xlabel('Средняя цена')
plt.ylabel('Количество отзывов')
plt.show()

немного коррелярует, если небольшая вероятность, что если больше цена, то отзывов меньше

4. Давайте рассмотрим кейс, в котором имена хозяина недвижимости и гостя совпадают. Как зовут хозяйку, у которой было в качестве гостей больше всего тёзок? Что все её гости отмечали в качестве плюсов сдаваемой недвижимости?

In [35]:
listings['host_name'] = listings['host_name'].str.lower()
reviews['reviewer_name'] = reviews['reviewer_name'].str.lower()
a = reviews.merge(listings, left_on='listing_id', right_on='id', how='left')
a['tezka'] = (a['host_name'] == a['reviewer_name'])
cnt = a.groupby('host_id').agg({'tezka': 'sum'}).reset_index().rename(columns={'tezka': 'cnt'})
result = cnt['cnt'].max()
host_with_most_tezochki = cnt[cnt['cnt'] == result]['host_id'].iloc[0]
host_name = listings[listings['host_id'] == host_with_most_tezochki]['host_name'].iloc[0]
print('Хозяйка с наибольшим количеством тёзок:', host_name, 'количество тёзок:', result)
positive_reviews = a[(a['host_name'] == a['reviewer_name']) & (a['comments'].notna())]['comments']
print("Плюсы отмеченные гостями: ", positive_reviews.unique())

Хозяйка с наибольшим количеством тёзок: robert количество тёзок: 3
Плюсы отмеченные гостями:  ['My Host made me feel very welcomed. Easy kitchen, to share. House rules, were  simple to follow. Modest accommodations, however, close to the Faulkner Hospital (15 minutes) by car, or within, walking distance. My room had air-conditioning, television, a desk and the bed was comfortable. \r\n~Lisa '
 "I had a great stay at Jason's place! The room was very clean and comfortable, and the building was quiet. It felt like home. There is a subway stop nearby (5-10 minute walk), and from there you can get to anywhere in the city. Plenty of restaurants, bars, and corner stores in the area too. Jason was friendly and an overall great host.  I highly recommend his place and I'd definitely stay there again."
 'Sara was very nice and accommodating! The studio was clean and well kept, with all needed amenities.'
 "Location was very convenient: quiet street, close to a beautiful park with wonderful view o